In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
df = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
# df_test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')


In [ ]:
df = df.drop(columns = 'Id')


In [ ]:
x = df.drop(columns = 'Cover_Type')
y = df.Cover_Type


In [ ]:
##create folds
from sklearn import model_selection
df["kfold"] = -1

df = df.sample(frac=1).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=10, shuffle=False)

for fold, (train_idx, val_idx) in enumerate(kf.split(X=df, y=df.Cover_Type.values)):
    print(len(train_idx), len(val_idx))
    df.loc[val_idx, 'kfold'] = fold


In [ ]:
import joblib
# import lightgbm as lgb
import xgboost as xg
import pandas as pd
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
def run(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    df_train = df_train.drop(columns = 'kfold')
    df_valid = df_valid.drop(columns = 'kfold')
    x_train = df_train.drop('Cover_Type', axis=1).values
    y_train = df_train.Cover_Type.values
    x_valid = df_valid.drop('Cover_Type', axis=1).values
    y_valid = df_valid.Cover_Type.values
    clf = xg.XGBClassifier(tree_method='gpu_hist',
                             num_class = 7,
                             learning_rate=0.09002114369564520,
                             gamma = 0.4893634098408942,
                             min_child_weight = 3.5,
                             subsample = 0.7,
                             colsample_bytree = 0.68973654,
                             reg_alpha = 16,
                             reg_lambda = 54,
                          )
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_valid)
    accuracy_score = metrics.accuracy_score(y_valid,y_pred)
    print(f"Fold={fold}, accuracy_score={accuracy_score}")
    File_name = 'model_xgb' + str(fold)
    joblib.dump(
    clf,File_name)
for i in range(10):
    run(fold = i)


In [ ]:
model_0_xgb= joblib.load('./model_xgb0')
model_1_xgb =joblib.load('./model_xgb1')
model_2_xgb= joblib.load('./model_xgb2')
model_3_xgb= joblib.load('./model_xgb3')
model_4_xgb= joblib.load('./model_xgb4')
model_5_xgb= joblib.load('./model_xgb0')
model_6_xgb =joblib.load('./model_xgb1')
model_7_xgb= joblib.load('./model_xgb2')
model_8_xgb= joblib.load('./model_xgb3')
model_9_xgb= joblib.load('./model_xgb4')


In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')
df_test = df_test.drop(columns = 'Id')


In [ ]:
model_0_xgb.classes_


In [ ]:
y_final_3_xgb = model_3_xgb.predict_proba(df_test)
y_final_0_xgb = model_0_xgb.predict_proba(df_test)
y_final_1_xgb = model_1_xgb.predict_proba(df_test)
y_final_2_xgb = model_2_xgb.predict_proba(df_test)
y_final_4_xgb = model_4_xgb.predict_proba(df_test)
y_final_5_xgb = model_6_xgb.predict_proba(df_test)
y_final_7_xgb = model_7_xgb.predict_proba(df_test)
y_final_8_xgb = model_8_xgb.predict_proba(df_test)
y_final_9_xgb = model_9_xgb.predict_proba(df_test)
y_final_6_xgb = model_6_xgb.predict_proba(df_test)


In [ ]:
y_final_avg = (y_final_0_xgb + y_final_1_xgb +y_final_2_xgb + y_final_3_xgb + y_final_4_xgb +
               y_final_5_xgb + y_final_6_xgb +y_final_7_xgb + y_final_8_xgb + y_final_9_xgb)/10


In [ ]:
y_final_sub = model_0_xgb.predict(df_test)
y_final_avg = y_final_avg.tolist()
for i in range(len(y_final_sub)):
    y_final_sub[i] = y_final_avg[i].index(max(y_final_avg[i]))


In [ ]:
y_final_sub = y_final_sub+1
print(y_final_sub)


In [ ]:
submission['Cover_Type'] = y_final_sub


In [ ]:
submission.to_csv('sub2.csv',index = False)
